Question: What topics are being talked about on Twitter about Theresa May and Boris Johnson the year before UK exits EU? 
          I am expecting a lot of similarities here, however I would like to refer to these for specific points in time to explain the                   sentiment analysis plot.

In [206]:
import pandas as pd

from pprint import pprint

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','user', 'com', 'co', 'con', 'be', 'else', 'http', 'would','send', 
                   'do', 'try', 'tell', 'go', 'get', 'can', 'think', 'know', 'give', 'ask', 
               'next', 'find'])

In [207]:
#https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [208]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Getting the dataset

In [209]:
cdf = pd.read_csv('cdf.csv')

In [211]:
cdf['PM'].value_counts()/len(cdf)

none             0.901269
Boris Johnson    0.053743
Theresa May      0.044988
Name: PM, dtype: float64

In [212]:
#reducing dataset from 300k to 50k (removing 90% of the none values)
#cdf = cdf.drop(cdf[cdf['PM'] == 'none'].sample(frac=.98).index)
df = cdf.drop(cdf[(cdf.PM == 'none')].index)

In [222]:
df_m = df[df['PM'] == 'Theresa May']
df_b = df[df['PM'] == 'Boris Johnson']
df_tm = df_m['TWEET_CLEANED'].tolist()
df_bj = df_b['TWEET_CLEANED'].tolist()

## Preparing the text

### Theresa May

### Creating a dictionary out of bigrams

In [223]:
bigram = gensim.models.Phrases(df_tm, min_count=1, threshold=1) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [256]:
#remove Stop Words
data_words_nostops = remove_stopwords(df_tm)
#creating bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
#Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ'])

In [257]:
print(data_lemmatized[:1][0][:30])

['colleague', 'civil', 'service', 'supported', 'referendum', 'damage', 'limitation', 'exercise', 'golden', 'opportun']


In [258]:
# Create Dictionary from the unique bigrams 
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=10)#, no_above=0.5) #excluding tokens that ocurred in less than 10 tweets and bigrams that occurred in more than 50% of the tweets
# Rebuild corpus based on the dictionary
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

### Using TF-IDF

#### A problem with this approach is that highly frequent words start to dominate in the document, but may not be representative to the model as less-frequent words. One way to fix this is to measure how unique (or how infrequent) a word is across all documents (or tweets), which is called the “inverse document frequency” or IDF. By introducing IDF, frequent words that are also frequent across all documents get penalized with less weight.
https://towardsdatascience.com/topic-modeling-of-2019-hr-tech-conference-twitter-d16cf75895b6

In [259]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

## Topic Modeling Via LDA

In [260]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [261]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.074*"vote" + 0.058*"tory" + 0.052*"time" + 0.047*"delay" + 0.041*"labour" '
  '+ 0.031*"country" + 0.028*"year" + 0.023*"voter" + 0.023*"referendum" + '
  '0.019*"day"'),
 (1,
  '0.190*"brexit" + 0.149*"deal" + 0.072*"people" + 0.021*"support" + '
  '0.016*"choice" + 0.015*"final" + 0.014*"negotiator" + 0.014*"majority" + '
  '0.013*"public" + 0.013*"trade"'),
 (2,
  '0.079*"party" + 0.059*"election" + 0.045*"good" + 0.042*"talk" + '
  '0.039*"leader" + 0.023*"leave" + 0.019*"many" + 0.018*"late" + '
  '0.016*"politician" + 0.016*"line"'),
 (3,
  '0.077*"conservative" + 0.039*"government" + 0.028*"hard" + 0.026*"remainer" '
  '+ 0.024*"thing" + 0.021*"today" + 0.021*"bad" + 0.021*"true" + '
  '0.018*"voting" + 0.018*"member"'),
 (4,
  '0.253*"theresa" + 0.043*"plan" + 0.036*"long" + 0.030*"parliament" + '
  '0.025*"news" + 0.024*"last" + 0.023*"british" + 0.022*"agreement" + '
  '0.018*"week" + 0.013*"second"')]


In [262]:
for idx, topic in lda_model.show_topics(formatted=False, num_words= 10):
    print('Topic: {} \nWords: {}'.format(idx, '|'.join([w[0] for w in topic])))

Topic: 0 
Words: vote|tory|time|delay|labour|country|year|voter|referendum|day
Topic: 1 
Words: brexit|deal|people|support|choice|final|negotiator|majority|public|trade
Topic: 2 
Words: party|election|good|talk|leader|leave|many|late|politician|line
Topic: 3 
Words: conservative|government|hard|remainer|thing|today|bad|true|voting|member
Topic: 4 
Words: theresa|plan|long|parliament|news|last|british|agreement|week|second


In [247]:
'''
Topic 18: Crisis
Topic 29: Petition
Topic 12: Referendum
'''

'\nTopic 18: Crisis\nTopic 29: Petition\nTopic 12: Referendum\n'

### Checking Coherence Score

In [263]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.33488365719019264


In [253]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])

df1 = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P'])

In [255]:
df1

,Topic,Word,P
0,0,head,0.472365
1,0,carry,0.159459
2,0,trash,0.084023
3,0,reply,0.049236
4,0,pointless,0.000150
...,...,...,...
295,29,conservative,0.000046
296,29,stop,0.000046
297,29,delay,0.000046
298,29,win,0.000046


In [242]:
lda_data =  pyLDAvis.gensim.prepare(lda_model, tfidf_corpus, id2word, mds='mmds')
pyLDAvis.display(lda_data)

/Users/ironhack/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
